# Survival Analysis: Which CTA L trains are most durable?

Data Strategy

1. Import Dataset
2. Clean Dataset
    * lowercase all text
    * create function that identifies the train line associated with the tweet.
    * create function that identifies the type of delay being reported.
    * extract the delay type using regular expressions to search for brackets.
    * extract the train line associated with the tweet.
    * filter dataframe to tweets where it involves a train line and some type of delay.


3. Prepare dataset for survival analysis
    * NEED TO READ MORE ABOUT SURVIVAL ANALYSIS....



In [57]:
import pandas as pd
import re

cta_tweets = pd.read_feather('cta_tweets_sample.feather')
cta_tweets['text'] = cta_tweets.text.str.lower()

print(cta_tweets.head())
cta_tweets.info()

           created_at                                               text  \
0 2018-07-09 20:42:01  some forest park-bound blue line trains are op...   
1 2018-07-09 20:51:29  forest park-bound blue line trains are standin...   
2 2018-07-09 20:52:46  some forest park-bound blue line trains are op...   
3 2018-07-09 21:09:15  loop-bound orange line trains are standing nea...   
4 2018-07-09 21:11:09  loop-bound orange line trains are standing nea...   

   retweet_count  favorite_count  
0              2               1  
1              2               1  
2              2               1  
3              0               1  
4              1               2  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5418 entries, 0 to 5417
Data columns (total 4 columns):
created_at        5418 non-null datetime64[ns]
text              5418 non-null object
retweet_count     5418 non-null int64
favorite_count    5418 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 169.

/Users/bhoeft/anaconda3/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [58]:
temp = cta_tweets.text[2]
print(temp)
re.search('delay', temp)

some forest park-bound blue line trains are operating with residual delays after an earlier mechanical problem near… https://t.co/iybgfseaix


<_sre.SRE_Match object; span=(68, 73), match='delay'>

In [105]:
def find_train_delays(x):
    """identifies if the tweet concerned a train,
    which was delayed|standing due to some problem,
    and 'crews are working to restore service'. These 
    are the typical original train delay tweets."""
    
    pattern = re.compile('(blue|brown|green|orange|purple|pink|red|yellow).*train.*(delay|standing|problem).*crews.*working')
    match = re.search(pattern, x) # match objects are bool=True
    
    if match:
        # return tuple of bool and the name of the train line that was delayed.
        return True, match.group(1)
    else:
        return False, None

    

In [107]:
print(cta_tweets.text[531])
temp2 = cta_tweets.text[531]

pattern = re.compile('(blue|brown|green|orange|purple|pink|red|yellow).*train.*(delay|standing|problem).*crews.*working')
match = find_train_delays(temp2)
print(match)

type(match)

update: orange line trains are standing in both directions at pulaski due to a door problem; crews working to restore service.
(True, 'orange')


tuple

Apply the `find_train_delays()` function to the *text* column.

In [ ]:
### cta_tweets.text.apply(find_train_delays)

In [45]:
for i in range(3200):
    print(cta_tweets.text[i])


some forest park-bound blue line trains are operating with residual delays after an earlier mechanical problem near… https://t.co/2q35suseoe
forest park-bound blue line trains are standing near clark/lake due to a mechanical problem; crews working to restore service.
some forest park-bound blue line trains are operating with residual delays after an earlier mechanical problem near… https://t.co/iybgfseaix
loop-bound orange line trains are standing near roosevelt due to a mechanical problem; crews working to restore service.
loop-bound orange line trains are standing near midway due to a mechanical problem; crews working to restore service.
some loop-bound orange line trains are operating with residual delays after an earlier mechanical problem near roos… https://t.co/f67gfwylij
some loop-bound orange line trains are operating with residual delays after an earlier mechanical problem near midway; service is resuming.
o'hare-bound blue line trains are standing at damen due to a mechanical

harlem-bound green line  trains are operating with residual delays after [an earlier door problem on a train at kedzie; service is resuming.
eastbound 75 74th-75th buses are temporarily rerouted via 75th, vincennes, 76th, and lafayette.
forest park-bound blue line trains are standing at racine due to a mechanical problem on a train; crews working to restore service.
forest park-bound blue line trains are operating with residual delays after an earlier mechanical problem on a trai… https://t.co/t064jpei6x
@bmthompson333 hi, brandon. sending trains express, as we noted, is a way to distribute capacity after something ca… https://t.co/jagcwp3m28
rt @chicagosmayor: starting soon, we're opening the new illinois medical district @cta blue line station. watch live at approx 11am: https:…
rt @chicagosmayor: #dyk: the cta has made unprecedented investments in rebuilding and rehabilitating rail stations. since 2011, more than 4…
rt @chicagosmayor: the illinois medical district has a new blue lin